In [11]:
import math
import sympy

In [292]:
def mh(N):
    return np.float64(pow(N, dvc))

In [293]:
def vcbound(N):
    return math.sqrt(8 / N * math.log(4 * mh(2 * N) / delta))

### 3.

In [294]:
dvc = 10
delta = 0.05
epsilon = 0.05

In [295]:
print(mh(10000))

1e+40


In [296]:
print(vcbound(420000))

0.05178593269970576


In [297]:
print(vcbound(440000))

0.050678810077732374


In [298]:
print(vcbound(460000))

0.04964277890917069


In [299]:
print(vcbound(480000))

0.04867047569610589


In [300]:
print(vcbound(500000))

0.047755566329267474


### 4.5.

In [301]:
dvc=50

In [302]:
print(vcbound(10000))
print(vcbound(5))

0.632174915200836
13.828161484991483


In [303]:
def rpb(N):
    return math.sqrt(2 / N * math.log(2 * N * mh(N))) + math.sqrt(2 / N / math.log(1 / delta)) + 1 / N

In [304]:
print(rpb(5))
print(rpb(10000))

6.3195192778301434
0.31500209730790407


In [305]:
def pv(N):
    y = sympy.Symbol('y')
    res = sympy.solve(y - sympy.sqrt(1 / N * (2 * y + sympy.log(6 * mh(2 * N) / delta))), y)
    return res

In [306]:
print(pv(5))
print(pv(10000))

[5.10136198199000]
[0.223698293680786]


In [311]:
def d(N):
    y = sympy.Symbol("y")
    f = y - sympy.sqrt(1 / 2 / N * (4 * y * (1+y) + sympy.log(4 / delta) + 200))
    res = sympy.solve(f, y)
    return res

In [312]:
print(d(5))
print(d(10000))

[6.17925346765694]
[0.101199750789236]


In [257]:
def vvcbound(N):
    return math.sqrt(16 / N * math.log(2 * mh(N) / math.sqrt(delta)))

In [258]:
print(vvcbound(5))
print(vvcbound(10000))

16.264111061012045
0.8604259707062739


### 16~20

In [140]:
import numpy as np

In [141]:
def sign(x):
    return ((x > 0).astype(np.int32) - 0.5) * 2

In [213]:
N = 20
s = 1
c = 3000

In [214]:
def generate_data(seed):
    np.random.seed(seed)
    x = np.random.randn(N).reshape(-1, 1)
    noise = np.random.randint(10, size=N).reshape(-1, 1)
    noise = (noise < 2).astype(np.int32)
    y = sign(x)
    noise = -2 * noise * y
    y = y + noise
    return np.hstack((x, y))

In [215]:
# data_ = generate_data(19)
# print(data_)
# data_ = data_[data_[:,0].argsort()]
# print(data_)
# data_ = np.vstack(([0, 0], data_))
# print(data_)

In [216]:
E_out_total = 0
E_in_total = 0

for i in range(c):
    data_ = generate_data(i + 70000)
    data_ = data_[data_[:,0].argsort()]
    data_ = np.vstack(([data_[0][0] - 0.00002, 0], data_))
    
    E_in = 1000000
    E_out = 0
    n_in = 0
    
    
    for j in range(data_.shape[0]):
        n = data_[j][0] + 0.00001
        a = ((data_[1:, 0] > n).astype(np.int32) - 0.5) * 2.0
        e = np.mean((a != data_[1:, 1]).astype(np.int32))
        if e < E_in:
            E_in = e
            n_in = n
            
    for j in range(data_.shape[0]):
        n = data_[j][0] + 0.00001
        a = ((data_[1:, 0] <= n).astype(np.int32) - 0.5) * 2.0
        e = np.mean((a != data_[1:, 1]).astype(np.int32))
        if e < E_in:
            E_in = e
            n_in = n
    
    E_out = 0.5 + 0.3 * (abs(n_in * 1.0) - 1)
    
    E_out_total = E_out_total + E_out
    E_in_total = E_in_total + E_in
    
E_out_total /= c
E_in_total /= c

In [217]:
print(E_in_total)
print(E_out_total)

0.1705833333333324
0.28890012604268284


### 19~20

In [219]:
import pandas as pd

In [220]:
data = pd.read_csv(
    "hw2_train.dat", 
    header=None, 
    sep=" "
)

In [221]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,NaN,8.105,-3.500,4.769,4.541,-9.829,5.252,3.838,-3.408,-4.824,-1
1,NaN,-6.273,-2.097,9.404,1.143,3.487,-5.206,0.061,5.024,-6.687,1
2,NaN,1.624,-1.173,4.260,-3.607,-6.632,4.431,-8.355,7.206,-8.977,1
3,NaN,-10.000,7.758,-2.670,-8.880,-1.099,-9.183,-4.086,8.962,5.841,1
4,NaN,8.464,1.762,2.729,2.724,8.155,6.096,-2.844,9.800,3.302,-1


In [224]:
train = data.iloc[:,1:]

In [225]:
print(train.values.shape)

(100, 10)


In [261]:
E_out_total = 0
E_in_total = 0
N = train.values.shape[0]
dim = train.values.shape[1] - 1
res = np.zeros((1, dim))
E_in = np.zeros((1, dim))

xx = train.values
for k in range(dim):
    xx = xx[xx[:, k].argsort()]
    row = np.zeros((1, dim + 1))
    row[0, k] = xx[0, k] - 0.00002
    xx = np.vstack((row, xx))

    E_in[0, k] = 1000000

    for j in range(N + 1):
        n = xx[j, k] + 0.00001
        a = ((xx[1:, k] > n).astype(np.int32) - 0.5) * 2.0
        e = np.mean((a != xx[1:, -1]).astype(np.int32))
        if e < E_in[0, k]:
            E_in[0, k] = e
            res[0, k] = n
            sign = 1

    for j in range(N + 1):
        n = xx[j, k] + 0.00001
        a = ((xx[1:, k] <= n).astype(np.int32) - 0.5) * 2.0
        e = np.mean((a != xx[1:, -1]).astype(np.int32))
        if e < E_in[0, k]:
            E_in[0, k] = e
            res[0, k] = n
            sign = -1
idx = np.argmax(E_in, axis=1)
print(E_in)
print(res)
print(idx)
print(sign)

[[0.39       0.3960396  0.41176471 0.27184466 0.45192308 0.31428571
  0.37735849 0.42990654 0.40740741]]
[[ 3.87501 -3.38999 -8.50899  1.46101 -9.62199  4.28801  3.95401 -2.87499
  -0.28599]]
[4]
-1


In [262]:
test = pd.read_csv(
    "hw2_test.dat", 
    header=None, 
    sep=" "
)

In [263]:
test = test.iloc[:, 1:]
test = test.values
print(test[:5, :])

[[ 0.531 -1.884 -0.351 -1.796 -9.891  6.12   2.486  8.44  -5.123 -1.   ]
 [ 5.123  5.047  5.404 -1.742 -0.317  9.585 -4.016 -1.8   -5.633  1.   ]
 [ 3.286  4.251 -4.837 -7.065 -7.546 -4.727  9.055  4.941 -6.287  1.   ]
 [-0.795 -1.617 -8.414 -5.391  6.641  1.269 -5.806 -7.375  9.469  1.   ]
 [-4.362  1.49  -7.232  0.802  4.424 -4.777  6.075  3.48  -9.837  1.   ]]


In [264]:
a = ((test[:, idx-1] <= res[0, idx-1]).astype(np.int32) - 0.5) * 2.0
e = np.mean((a != test[:, -1]).astype(np.int32))
print(e)

0.49538


In [272]:
import sys
import numpy as np
import math
from random import *

def read_input_data(path):
    x = []
    y = []
    for line in open(path).readlines():
        items = line.strip().split(' ')
        tmp_x = []
        for i in range(0,len(items)-1): tmp_x.append(float(items[i]))
        x.append(tmp_x)
        y.append(float(items[-1]))
    return np.array(x),np.array(y)

def calculate_Ein(x,y):
    # calculate median of interval & negative infinite & positive infinite
    thetas = np.array( [float("-inf")]+[ ( x[i]+x[i+1] )/2 for i in range(0, x.shape[0]-1) ]+[float("inf")] )
    Ein = x.shape[0]
    sign = 1
    target_theta = 0.0
    # positive and negative rays
    for theta in thetas:
        y_positive = np.where(x>theta,1,-1)
        y_negative = np.where(x<theta,1,-1)
        error_positive = sum(y_positive!=y)
        error_negative = sum(y_negative!=y)
        if error_positive>error_negative:
            if Ein>error_negative:
                Ein = error_negative
                sign = -1
                target_theta = theta
        else:
            if Ein>error_positive:
                Ein = error_positive
                sign = 1
                target_theta = theta
    return Ein, target_theta, sign

def main():
#     x,y = read_input_data("train.dat")
    x = train.values[:, :-1]
    y = train.values[:, -1]
    # record optimal descision stump parameters
    Ein = x.shape[0]
    theta = 0
    sign = 1
    index = 0
    # multi decision stump optimal process
    for i in range(0,x.shape[1]):
        input_x = x[:,i]
        input_data = np.transpose(np.array([input_x,y]))
        input_data = input_data[np.argsort(input_data[:,0])]
        curr_Ein,curr_theta,curr_sign = calculate_Ein(input_data[:,0],input_data[:,1])
        if Ein>curr_Ein:
            Ein = curr_Ein
            theta = curr_theta
            sign = curr_sign
            index = i
    print("1:", Ein*1.0/x.shape[0])
    # test process
#     test_x,test_y = read_input_data("test.dat")
#     test_x = test_x[:,index]
    test_x = test[:, :-1]
    test_y = test[:, -1]
    predict_y = np.array([])
    if sign==1:
        predict_y = np.where(test_x>theta,1.0,-1.0)
    else:
        predict_y = np.where(test_x<theta,1.0,-1.0)
    Eout = np.sum(predict_y!=test_y)
    print (Eout*1.0/test_x.shape[0])
    
main()

1: 0.25
0.001


/usr/local/Cellar/ipython/6.4.0/libexec/vendor/lib/python3.6/site-packages/ipykernel_launcher.py:72: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.
